In [1]:
# Dependencies and Setup
import json
import pandas as pd
import requests
import numpy as py
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine, inspect, func, distinct
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import sqlite3

In [2]:
from sqlite3 import Error

In [3]:
engine = create_engine("sqlite:///../SqLIte_files/NBAfantasyML.sqlite")
conn = engine.connect()

In [4]:
# create function to try and connect to a sqlite database.  if it doesn't work, create one...
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [5]:
if __name__ == '__main__':
    create_connection("../SqLIte_files/NBAfantasyML.sqlite")

2.6.0


In [6]:
# Bring in data as object jasonified
player_name_personID_obj=requests.get("http://data.nba.net/prod/v1/2018/players.json")
player_name_personID_request=player_name_personID_obj.json()

In [7]:
# Loop through data and pull out data

NBA_player_list=[]
for players in player_name_personID_request["league"]["standard"]:
    player_list_dict={}
    player_list_dict["personId"]=players["personId"]
    player_list_dict["lastName"]=players["lastName"]
    player_list_dict["firstName"]=players["firstName"]
    
# Append NBA_player_list= personID 
    NBA_player_list.append(player_list_dict["personId"])

In [8]:
NBA_player_list# Count # of Players
player_response_count=player_name_personID_request["league"]["standard"]
player_count=len(player_response_count)
player_count

498

In [9]:
NBA_player_list_df=pd.DataFrame(data=NBA_player_list)
NBA_player_list_df.head()

,0
0,203518
1,203112
2,1629121
3,203500
4,1628389


In [10]:
Player_stats_yr=[{}]

In [11]:
for i in range(0,player_count):
    number_chosen_lst=i
    URL1="http://data.nba.net/prod/v1/2018/players/"
    URL2=NBA_player_list[number_chosen_lst]
    URL3="_profile.json"
    URL_pull=URL1+URL2+URL3
    player_response_obj=requests.get(URL_pull)
    player_response=player_response_obj.json()

    # Count # of Players
    # player_response_count=player_response["league"]["standard"]["stats"]["regularSeason"]["season"]
    # player_response_count
    Seasons_needed=[2015,2016,2017]
    for season_test in player_response["league"]["standard"]["stats"]["regularSeason"]["season"]:
        season_dict = {}
        season_dict["seasonYear"] = season_test["seasonYear"]
        season_dict["ppg"]= season_test["total"]["ppg"]
        season_dict["rpg"]= season_test["total"]["rpg"]
        season_dict["apg"]= season_test["total"]["apg"]
        season_dict["mpg"]= season_test["total"]["mpg"]
        season_dict["topg"]= season_test["total"]["topg"]
        season_dict["spg"]= season_test["total"]["spg"]
        season_dict["bpg"]= season_test["total"]["bpg"]
        season_dict["tpp"]= season_test["total"]["tpp"]
        season_dict["ftp"]= season_test["total"]["ftp"]
        season_dict["fgp"]= season_test["total"]["fgp"]
        season_dict["assists"]= season_test["total"]["assists"]
        season_dict["blocks"]= season_test["total"]["blocks"]
        season_dict["steals"]= season_test["total"]["steals"]
        season_dict["turnovers"]= season_test["total"]["turnovers"]
        season_dict["offReb"]= season_test["total"]["offReb"]
        season_dict["defReb"]= season_test["total"]["defReb"]
        season_dict["totReb"]= season_test["total"]["totReb"]
        season_dict["fgm"]= season_test["total"]["fgm"]
        season_dict["fga"]= season_test["total"]["fga"]
        season_dict["tpm"]= season_test["total"]["tpm"]
        season_dict["tpa"]= season_test["total"]["tpa"]
        season_dict["ftm"]= season_test["total"]["ftm"]
        season_dict["fta"]= season_test["total"]["fta"]
        season_dict["pFouls"]= season_test["total"]["pFouls"]
        season_dict["points"]= season_test["total"]["points"]
        season_dict["gamesPlayed"]= season_test["total"]["gamesPlayed"]
        season_dict["gamesStarted"]= season_test["total"]["gamesStarted"]
        season_dict["plusMinus"]= season_test["total"]["plusMinus"]
        season_dict["min"]= season_test["total"]["min"]
        season_dict["dd2"]= season_test["total"]["dd2"]
        season_dict["td3"]= season_test["total"]["td3"]

    #     Append Player_stats_year
        Player_stats_yr.append({"personId":URL2,
                                "seasonYear": season_dict["seasonYear"],
                                "PTG": season_dict["ppg"],
                                "TRB": season_dict["rpg"],
                                "AST": season_dict["apg"],
                                "mpg": season_dict["mpg"],
                                "TOV": season_dict["topg"],
                                "STL": season_dict["spg"],
                                "BLK": season_dict["bpg"],
                                "tpp": season_dict["tpp"],
                                "FTP": season_dict["ftp"],
                                "FGP": season_dict["fgp"],
                                "assists": season_dict["assists"],
                                "blocks": season_dict["blocks"],
                                "steals": season_dict["steals"],
                                "turnovers": season_dict["turnovers"],
                                "offReb": season_dict["offReb"],
                                "defReb": season_dict["defReb"],
                                "totReb": season_dict["totReb"],
                                "FG": season_dict["fgm"],
                                "FGA": season_dict["fga"],
                                "TPP": season_dict["tpm"],
                                "tpa": season_dict["tpa"],
                                "FT": season_dict["ftm"],
                                "FTA": season_dict["fta"],
                                "pFouls": season_dict["pFouls"],
                                "points": season_dict["points"],
                                "gamesPlayed": season_dict["gamesPlayed"],
                                "gamesStarted": season_dict["gamesStarted"],
                                "plusMinus": season_dict["plusMinus"],
                                "min": season_dict["min"],
                                "dd2": season_dict["dd2"],
                                "td3": season_dict["td3"]})

In [12]:
playerYearlyStatsAllYearsdf=pd.DataFrame(Player_stats_yr)
player_yr_stats=playerYearlyStatsAllYearsdf[["personId","seasonYear","FG","FGA","FGP","TPP","FT","FTP","TRB",
                                              "AST","STL","BLK","TOV","PTG"]]
player_yr_stats

,personId,seasonYear,FG,FGA,FGP,TPP,FT,FTP,TRB,AST,STL,BLK,TOV,PTG
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,203112,2018.0,4,18,22.2,2,7,70,2.5,0.8,0.1,0.4,0.4,1.7
2,203112,2017.0,130,365,35.6,102,49,81.7,3.7,0.8,0.5,0.4,0.9,5.9
3,203112,2016.0,,,0,,,0,0,0,0,0,0,0
4,203112,2015.0,119,214,55.6,19,50,73.5,3.2,0.5,0.5,0.4,0.5,5.2
5,203112,2014.0,152,331,45.9,18,76,78.4,4.4,1,0.4,0.3,0.9,5.9
6,203112,2013.0,,,0,,,0,0,0,0,0,0,0
7,203112,2012.0,42,75,56,1,31,81.6,2.7,0.4,0.4,0.5,0.6,4
8,1629121,2018.0,3,10,30,2,0,0,0.3,0.7,0.3,0.1,0.1,0.9
9,203500,2018.0,304,497,61.2,0,115,56.1,10,1.7,1.5,0.8,1.6,15.4


In [13]:
# Load Yearly Players Stats to SqLite
player_yr_stats.to_sql('Yearly_Player_Stats',conn, if_exists='replace', index=False)

OperationalError: (sqlite3.OperationalError) too many SQL variables [SQL: 'INSERT INTO "Yearly_Player_Stats" ("personId", "seasonYear", "FG", "FGA", "FGP", "TPP", "FT", "FTP", "TRB", "AST", "STL", "BLK", "TOV", "PTG") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?), (?, ?, ?, ?, ?, ?, ?, ?, ?,